In [3]:
from dotenv import load_dotenv
import os 
import pymongo

from enum import Enum
from pydantic import BaseModel 
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

from langchain_core.pydantic_v1 import BaseModel, Field

load_dotenv()

False

In [ ]:
class DrinkTaste(str, Enum):
    SWEET = "Sweet"
    SOUR = "Sour"
    BITTER = "Bitter"
    SALTY = "Salty"
    UMAMI = "Umami"
    FRUITY = "Fruity"
    FLORAL = "Floral"
    SPICY = "Spicy"
    CREAMY = "Creamy"
    TART = "Tart"
    REFRESHING = "Refreshing"
    RICH = "Rich"
    LIGHT = "Light"
    EARTHY = "Earthy"
    CITRUSY = "Citrusy"
    HERBAL = "Herbal"
    
class TasteAnalysis(BaseModel):
    think: str = Field(description="Detailed reasoning for the taste analysis")
    tastes: list[DrinkTaste] = Field(description="Final dominant taste characteristics as a comma-separated list without any other words")
        

llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0.5, include_raw=True)
structured_llm = llm.with_structured_output(TasteAnalysis, method="json_schema")

In [25]:
client = pymongo.MongoClient(os.getenv('MONGO_CONNECTION_STRING'))
db = client["monin"]
collection = db["drinks"]

In [70]:
prompt = PromptTemplate.from_template(
    """Analyze the drink recipe and determine the dominant taste characteristics based on the ingredients and their interactions.
Taste options:
SWEET = "Sweet"
SOUR = "Sour"
BITTER = "Bitter"
SALTY = "Salty"
UMAMI = "Umami"
FRUITY = "Fruity"
FLORAL = "Floral"
SPICY = "Spicy"
CREAMY = "Creamy"
TART = "Tart"
REFRESHING = "Refreshing"
RICH = "Rich"
LIGHT = "Light"
EARTHY = "Earthy"
CITRUSY = "Citrusy"
HERBAL = "Herbal"

Recipe: {recipe}

Output a JSON object with the following structure:
{{
  "think": "Detailed reasoning for the taste analysis based on the ingredients. Reasoning about it, come up with a logical list of 2-4 flavors, and at the end, in a separate sentence, list them comma-separated and uncluttered.",
  "tastes": ["Pick 2-4 most suitable tastes from "think" part and print out them as a comma-separated list without any other words"]
}}

The "think" part should end with a specific list listed with commas. You get a paycheck for the right list, and a fine for the wrong one. If there is no comma-separated list at the end of the reflection part, you will be fired!

Make sure the "tastes" list contains only valid options from the provided list. And be careful, all tastes in "tastes" should be the same as in ending of "think" part.
AI:"""
)

for drink in collection.find({})[:20]:
    recipe = drink['name'] + "\n" + " ".join(drink['recipie'])

    tastes = structured_llm.invoke(prompt.format(recipe=recipe))
    
    if tastes:
        tastes = tastes.tastes
        tastes = [taste.value for taste in tastes]
        print(drink['name'])
        print(tastes)
    else:
        print(f"No tastes identified for {drink['name']}")
    print()


Hawaiian Pearl Bubble Tea
['Sweet', 'Sour', 'Salty', 'Tart', 'Creamy', 'Floral']

Blue Bubble Milk Tea
['Floral', 'Citrusy', 'Spicy']

Strawberry Bubble Burst Tea
['Sweet', 'Sour', 'Bitter', 'Citrusy']

Bubble Chai Tea
['Sweet', 'Creamy', 'Floral', 'Umami']

Golden Jelly Bubble Tea
['Umami', 'Salty', 'Floral']

Mango Milk Tea
['Sweet', 'Tart', 'Umami']

Boba Oat Milk Latte
['Sweet', 'Sour', 'Spicy', 'Tart']

Kiwi Honey Milk Tea
['Citrusy', 'Floral', 'Herbal', 'Sour', 'Spicy']

Black Berry Sugar Boba Milk Tea
['Citrusy', 'Floral', 'Bitter']

Black Sugar Boba Milk Tea
['Sweet', 'Creamy', 'Sour', 'Spicy', 'Umami', 'Floral']

Black Raven Milk Punch
['Citrusy', 'Herbal']

Tropical Guava Spritz
['Sweet', 'Tart', 'Floral']

Very Cherry Mojito
['Citrusy', 'Sour', 'Bitter']

Cranberry Aperol Spritz
['Sweet', 'Sour', 'Tart', 'Umami']

Low ABV Coffee Cocktail
['Floral', 'Sweet', 'Sour', 'Spicy', 'Tart', 'Umami']

Yuzu Lojito
['Citrusy', 'Sour', 'Sweet', 'Floral']

Pineberry Beer
['Sweet', 'Sour']